In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum

In [3]:
df = pd.read_csv("datos.csv", encoding="latin1")
df = df[df["ID"] > 60]
df.rename(columns={'Ethical technology assessment?':'Ethical technology assessment'}, inplace=True)

bach = df[df['program'] == 'bachelor']
mast = df[df['program'] == 'master']

b_aal = bach[bach['campus'] == 'AAL']
m_aal = mast[mast['campus'] == 'AAL']

b_cph = bach[bach['campus'] == 'CPH']
m_cph = mast[mast['campus'] == 'CPH']

esb = df[df['campus'] == 'ESB']

In [4]:
col1 = 'Select 3 workshops in a prioritised order, where 1 is the workshop you want to attend the most4'
col2 = 'Select 3 workshops in a prioritised order, where 1 is the workshop you want to attend the most5'
col3 = 'Select 3 workshops in a prioritised order, where 1 is the workshop you want to attend the most6'

cursos_b_aal = list(set(b_aal[col1]) | set(b_aal[col2]) | set(b_aal[col3]))
cursos_m_aal = list(set(m_aal[col1]) | set(m_aal[col2]) | set(m_aal[col3]))

pref_b_aal = b_aal[cursos_b_aal]
pref_m_aal = m_aal[cursos_m_aal]

In [53]:
def generate_col_nums(data):
    ans = {}
    c = 0
    for col in data.columns:
        ans[col] = c
        c += 1
    return ans

def get_dict(data, row):
    ans = {}
    # PROVISOR #
    ans[1] = 0
    ans[2] = 0
    ans[3] = 0
    names = generate_col_nums(data)
    for col in data.columns:
        if not np.isnan(data.iloc[row][col]):
            ans[int(data.iloc[row][col])] = names[col]
    return ans

get_dict(pref_m_aal, 1)

{1: 2, 2: 6, 3: 0}

In [26]:
pref_m_aal.head()

,Digital and hybrid practices in collaborative work,Psychological safety in team work,Creative and systems thinking for complex problem-solving (whole day workshop - you are then attending 1 workshop the whole day),Professional identity development,Clarification of individual professional competences,Ethical technology assessment,Professional communication with external partners,Leadership and project management
13,NaN,NaN,3.0,2.0,1.0,NaN,NaN,NaN
15,3.0,NaN,1.0,NaN,NaN,NaN,2.0,NaN
20,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN
21,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN
24,2.0,NaN,NaN,NaN,3.0,NaN,NaN,1.0


# Implementación Master Aalborg

In [85]:
n_students = len(pref_m_aal)
n_workshops = pref_m_aal.shape[1]

S = [n for n in range(n_students)]
T = [n for n in range(n_workshops)]
H = [0, 1]
C = [4, 4]
U = [50 for n in range(n_workshops)]
t_pref = [get_dict(pref_m_aal, n) for n in range(n_students)]
D = [0 for n in range(n_workshops)]

p = [3, 2, 1]

# Para saber preferencia de estudiante s en opcion i. se aplica t_pref[s][i]

In [86]:
for s in S:
    for i in range(3):
        print(s, i)
        print(t_pref[s][i + 1])

0 0
4
0 1
3
0 2
2
1 0
2
1 1
6
1 2
0
2 0
4
2 1
3
2 2
0
3 0
4
3 1
3
3 2
0
4 0
7
4 1
0
4 2
4
5 0
0
5 1
6
5 2
4
6 0
2
6 1
0
6 2
7
7 0
2
7 1
7
7 2
4
8 0
4
8 1
6
8 2
7
9 0
1
9 1
0
9 2
2
10 0
1
10 1
0
10 2
2
11 0
2
11 1
7
11 2
4
12 0
1
12 1
0
12 2
2
13 0
2
13 1
7
13 2
4
14 0
0
14 1
7
14 2
3
15 0
7
15 1
1
15 2
2
16 0
4
16 1
7
16 2
6
17 0
7
17 1
4
17 2
3
18 0
4
18 1
7
18 2
3
19 0
1
19 1
3
19 2
7
20 0
7
20 1
4
20 2
1
21 0
7
21 1
4
21 2
3
22 0
3
22 1
4
22 2
2
23 0
7
23 1
3
23 2
6
24 0
0
24 1
0
24 2
3
25 0
2
25 1
3
25 2
6
26 0
6
26 1
3
26 2
4
27 0
2
27 1
3
27 2
7
28 0
4
28 1
6
28 2
3
29 0
4
29 1
6
29 2
3
30 0
4
30 1
6
30 2
3
31 0
4
31 1
6
31 2
3
32 0
0
32 1
4
32 2
2
33 0
4
33 1
7
33 2
2
34 0
7
34 1
1
34 2
6
35 0
7
35 1
6
35 2
1
36 0
4
36 1
7
36 2
6
37 0
4
37 1
3
37 2
0
38 0
4
38 1
6
38 2
7
39 0
4
39 1
6
39 2
7
40 0
7
40 1
4
40 2
6
41 0
2
41 1
7
41 2
4
42 0
7
42 1
6
42 2
3
43 0
4
43 1
7
43 2
3
44 0
4
44 1
3
44 2
6
45 0
3
45 1
2
45 2
0
46 0
7
46 1
0
46 2
2
47 0
4
47 1
7
47 2
3
48 0
3
48 1
4
48 2
7
4

In [87]:
model = Model('m_aal')

###############
## VARIABLES ##
###############

z = model.addVars(T, vtype = GRB.BINARY, name = 'z')
y = model.addVars(T, H, vtype=GRB.BINARY, name="y")
w = model.addVars(S, T, vtype=GRB.BINARY, name="w")

###################
## RESTRICCIONES ##
###################

## Facility Location
for t in T:
    model.addConstr(quicksum(y[t, h] for h in H) == z[t], name=f"asign_horario[{t}]")

for h in H:
    model.addConstr(quicksum(y[t, h] for t in T) <= C[h], name=f"capacidad_horario[{h}]")

## Asignación Estudiantes
for s in S:
    model.addConstr(quicksum(w[s, t] for t in T) >= 1, name=f"asign_est[{s}]")

for t in T:
    model.addConstr(quicksum(w[s, t] for s in S) <= U[t] * z[t], name=f"cap_taller[{t}]")

## Asignación Horario REVISARRRRR
for s in S:
    for h in H:
        model.addConstr(quicksum(w[s, t] * y[t, h] for t in T) <= 1, name=f"conflicto_horario[{s},{h}]")

## Talleres Diarios REVISARRRRRR
for t in T:
    for h in H:
        for h2 in H:
            if h != h2:
                model.addConstr(y[t, h] + D[t]*y[t, h2] <= 2, name=f"diario[{t},{h},{h2}]")

######################
## FUNCIÓN OBJETIVO ##
######################

model.setObjective(quicksum(p[i] * quicksum(w[s, t_pref[s][i + 1]] for s in S) for i in range(3)), GRB.MAXIMIZE)

In [88]:
%%time
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 23.6.0 23H723)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 427 rows, 3168 columns and 6352 nonzeros
Model fingerprint: 0x703e7a4b
Model has 786 quadratic constraints
Variable types: 0 continuous, 3168 integer (3168 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 16 rows and 0 columns
Presolve time: 0.01s
Presolved: 7485 rows, 9456 columns, 31488 nonzeros
Variable types: 0 continuous, 9456 integer (9456 binary)
Found heuristic solution: objective 290.0000000
Found heuristic solution: objective 291.0000000

Root relaxation: objective 9.630000e+02, 2405 iterations, 0.04 seconds (0.10 work units)

    Nodes    |    Current Node    |     Objectiv

In [89]:
z

{0: <gurobi.Var z[0] (value 1.0)>,
 1: <gurobi.Var z[1] (value 1.0)>,
 2: <gurobi.Var z[2] (value 1.0)>,
 3: <gurobi.Var z[3] (value 1.0)>,
 4: <gurobi.Var z[4] (value 1.0)>,
 5: <gurobi.Var z[5] (value 1.0)>,
 6: <gurobi.Var z[6] (value 1.0)>,
 7: <gurobi.Var z[7] (value 1.0)>}